In [1]:
import pandas as pd
import numpy as np
import sympy as sp
import gurobipy as gp
from gurobipy import GRB
import sys
import os
import tqdm
import time

# Get the current script's directory
current_dir = os.getcwd()

# Navigate to the target directory one up and two down
target_dir = os.path.abspath(os.path.join(current_dir, '..', 'src'))

# Add the target directory to sys.path
sys.path.append(target_dir)

In [2]:
# Select random seed
np.random.seed(0)

#Generate one sector
x = sp.symbols('x')
dummy_market = pd.DataFrame({
    'id': 1,
    'name': 'dummy_market',
    'permit_price': [2] # the price is fixed and is the market-clrearing price 
    # If permit price < 2.3 then it crushes
    })
dummy_sector = pd.DataFrame({
  'id': 1,
    'name': 'dummy_sector',
    'price_demand_function': [10 - 0.1*x]
    })

#Genarate two firms

dummy_firm1 = pd.DataFrame({
    'id': 1,
    'name': 'dummy_firm1',
    'sector_id': 1,
    'country': 'US',
    'production_cost_function': [x*0],
    'abatement_cost_function': [5*x**2],
    'emission_to_output_ratio': 1,
    'actual_output': 0,
    'emission': 0,
    'free_emission_multiplier': [0.1],
    'profit': 0
    })
dummy_firm2 = pd.DataFrame({
    'id': 2,
    'name': 'dummy_firm2',
    'sector_id': 1,
    'country': 'US',
    'production_cost_function': [x],
    'abatement_cost_function': [x**2*0.01],
    'emission_to_output_ratio': 1,
    'actual_output': 0,
    'emission': 0,
    'free_emission_multiplier': [0.1],
    'profit': 0
    })

In [3]:
type((x)*0.5)

sympy.core.mul.Mul

In [5]:
import sympy as sp
from gurobipy import Model, LinExpr, QuadExpr, GRB

def sympy_to_gurobi(sympy_expr, symbol_map):
    """
    Recursively convert a SymPy expression to a Gurobi expression.
    
    Parameters:
        sympy_expr (sp.Expr): SymPy expression to convert.
        symbol_map (dict): Mapping from SymPy symbols to Gurobi variables.
        
    Returns:
        Gurobi expression (LinExpr, QuadExpr, or constant).
    """
    if isinstance(sympy_expr, sp.Symbol):
        return symbol_map[sympy_expr]
    
    elif isinstance(sympy_expr, sp.Add):
        return sum(sympy_to_gurobi(arg, symbol_map) for arg in sympy_expr.args)
    
    elif isinstance(sympy_expr, sp.Mul):
            result = 1
            for arg in sympy_expr.args:
                result *= sympy_to_gurobi(arg, symbol_map)
            return result
    
    elif isinstance(sympy_expr, sp.Pow):
        base, exp = sympy_expr.args
        if exp == 2:
            base_expr = sympy_to_gurobi(base, symbol_map)
            return QuadExpr(base_expr * base_expr)
        else:
            raise ValueError("Non-quadratic powers are not supported by Gurobi.")
    
    elif isinstance(sympy_expr, sp.Number):
        return float(sympy_expr)
    
    else:
        raise ValueError(f"Unsupported SymPy expression: {sympy_expr}")


# Example usage
x, y = sp.symbols('x y')
sympy_expr = +3*x - 2*y**2 - 4*y 

# Create a Gurobi model
model = Model("example")
x_gurobi = model.addVar(name="x", lb=0, vtype=GRB.CONTINUOUS, ub = 10)
y_gurobi = model.addVar(name="y", lb=0, vtype=GRB.CONTINUOUS, ub = 10)

# Map SymPy symbols to Gurobi variables
symbol_map = {x: x_gurobi, y: y_gurobi}

# Convert SymPy expression to Gurobi expression
gurobi_expr = sympy_to_gurobi(sympy_expr, symbol_map)

# Set the objective and optimize
model.setObjective(gurobi_expr, GRB.MAXIMIZE)
model.optimize()

# Print the results
for v in model.getVars():
    print(f"{v.varName}: {v.X}")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 2 columns and 0 nonzeros
Model fingerprint: 0xcc5870d7
Model has 1 quadratic objective term
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [3e+00, 4e+00]
  QObjective range [4e+00, 4e+00]
  Bounds range     [1e+01, 1e+01]
  RHS range        [0e+00, 0e+00]
Presolve removed 0 rows and 2 columns
Presolve time: 0.02s
Presolve: All rows and columns removed

Barrier solved model in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective 3.00000000e+01
x: 10.0
y: 0.0


In [6]:
import sympy as sp

def taylor_approx(expr, var_list, point_dict=None, order=2):
    """
    Approximates the given expression using the first three Taylor series terms (up to quadratic) for non-linear terms.

    Parameters:
    expr (sympy.Expr): The sympy expression to approximate.
    var_list (list): A list of sympy symbols representing the variables for Taylor expansion.
    order (int): The highest order term to include in the Taylor series (default is 2 for quadratic).
    
    Returns:
    sympy.Expr: The approximated expression.
    """
    approx_expr = expr
    for var in var_list:
        # Perform Taylor expansion around var=point up to the specified order (default is 2)
        taylor_series = sp.series(approx_expr, var, point_dict[var], order+1).removeO()
        approx_expr =  taylor_series
    
    return approx_expr

# Example usage:
x, y = sp.symbols('x y')
expr = sp.sin(x) + x**3 + y**4
vars_to_expand = [x, y]
points = {x: 1, y: 1}

approx_expr = taylor_approx(expr, vars_to_expand, points)
print(approx_expr)

4*y + (3 - sin(1)/2)*(x - 1)**2 + (x - 1)*(cos(1) + 3) + 6*(y - 1)**2 - 2 + sin(1)


In [7]:
def is_quadratic(expr, var_list): # Thelei allagi! Prepei sto trito paradeigma na girizei false!
    """
    Checks if the given sympy expression is quadratic in all specified variables.

    Parameters:
    expr (sympy.Expr): The sympy expression to check.
    var_list (list): A list of sympy symbols representing the variables.

    Returns:
    bool: True if the expression is quadratic, False otherwise.
    """
    for term in expr.as_ordered_terms():
        degree = max(term.as_poly(var).degree(var) for var in var_list if term.as_poly(var) is not None)
        #print(f" Degree: {degree}, Term: {term}")

        if degree > 2:
            return False
    return True

# Example usage:
x, y = sp.symbols('x y')
expr1 = x**2 + y**2 + x*y + x + y + 1  # Quadratic
expr2 = x**3 + y**2 + sp.exp(x) # Not quadratic

print(is_quadratic(expr1, [x, y]))  # Output: True
print(is_quadratic(expr2, [x, y]))  # Output: False

em, out = sp.symbols('em out')
exp = -5*em + out*(99.0 - 0.1*out) + out - 10*(-em + out)**2 + em**2*out
print(is_quadratic(exp, [em, out]))  

True
False
True


In [8]:
import sympy as sp
import gurobipy as gp
from gurobipy import GRB

# Assuming x is already defined as a symbol
x = sp.symbols('x')

# Print the function of abatement cost evaluated at x = 5, using sympy
print("Abatement cost function evaluated at 5:{}".format(dummy_firm1['abatement_cost_function'][0].subs(x, 5).evalf()))  # 74.20...

# Calculate the output of a firm taking the decisions of the rest of the firms as given
def calculate_output(firm, firms, dummy_market, dummy_sector, lp_counter = 0):
    # Calculate the output of the firm

    # Sum of all other outputs
    sum_other_outputs = 0
    for i in range(len(firms)):
        if firms[i]['id'][0] != firm['id'][0]:
            sum_other_outputs += firms[i]['actual_output'][0]
    
    #Get the price of the permits
    permit_price = dummy_market['permit_price'][0]
    # Get the price demand function of the sector
    price_demand_function = dummy_sector['price_demand_function'][0]
    # Get the abatement cost function of the firm
    abatement_cost_function = firm['abatement_cost_function'][0]
    # Get the production cost function of the firm
    production_cost_function = firm['production_cost_function'][0]
    # Get the free emission multiplier of the firm
    free_emission_multiplier = firm['free_emission_multiplier'][0]


    out, em = sp.symbols('out em')


    # Calculate the output of the firm


    #print("Price to demand Function: {}".format(price_demand_function.subs(x, sum_other_outputs + out)))
    income = (price_demand_function.subs(x, sum_other_outputs + out) - production_cost_function.subs(x, out))*out
    abatement = -abatement_cost_function.subs(x, out - em)
    trading = - permit_price * (em -free_emission_multiplier * out)
    profit_expr = income + abatement + trading
    print(f"The profit expr: {profit_expr}")
    # if(is_quadratic(profit_expr, [out, em])):
    #     #print("The profit expr is quadratic {}".format(profit_expr))
    #     second_order = profit_expr
    # else:
    #     second_order = taylor_approx(profit_expr, [out, em], {out :1, em :1}, order=2) #ALLAGI ME KANONIKA NOUMERA
    #     print(f"The profit expr: {profit_expr} \nis turned to: {second_order}")
    second_order = profit_expr
    m = gp.Model("firm")
    output = m.addVar(vtype=GRB.CONTINUOUS, name="output", lb=0)
    emission = m.addVar(vtype=GRB.CONTINUOUS, name="emission", lb=0)
    symbol_map = {out: output, em: emission}
    profit = sympy_to_gurobi(second_order, symbol_map)
    m.setObjective(profit, GRB.MAXIMIZE)
    gur_abatement = sympy_to_gurobi(abatement, symbol_map)
    m.addConstr(emission <= output)
    m.Params.LogToConsole = 0
    if lp_counter > 0:
        m.write("old{}.lp".format(lp_counter))
    m.optimize()


    # print("income: {}".format(income.subs(out, output.X).subs(em, emission.X)))
    # print("abatement: {}".format(abatement.subs(out, output.X).subs(em, emission.X)))
    # print("trading: {}".format(trading.subs(out, output.X).subs(em, emission.X)))
    # for v in m.getVars():
    #     print(f"{v.varName}: {v.X}")
    # return sum_other_outputs
    return output.X, emission.X, profit_expr.subs(out, output.X).subs(em, emission.X).evalf()

print(calculate_output(dummy_firm1, [dummy_firm1, dummy_firm2], dummy_market, dummy_sector))

Abatement cost function evaluated at 5:125.000000000000
The profit expr: -2*em + out*(10 - 0.1*out) + 0.2*out - 5*(-em + out)**2
(41.00000000009346, 40.79999999430374, 168.300000000000)


In [9]:
import math
firms = [dummy_firm1, dummy_firm2]

def optimize_them_all(firms, dummy_market, dummy_sector, print_output=True,print_diff=False):
    repeat = True
    
    while repeat:
        lp_counter = 1
        max_diff = 0
        repeat = False
        for firm in firms:
            output, emission, profit = calculate_output(firm, firms, dummy_market, dummy_sector, lp_counter= lp_counter)
            lp_counter += 1
            #print("Firm {} has 'output - firm['actual_output'][0]': {:5f} and 'emission - firm['emission'][0]': {:5f}".format(firm['name'][0], output - firm['actual_output'][0], emission - firm['emission'][0]))
            if abs(output - firm['actual_output'][0])>0.01 or abs(emission - firm['emission'][0])>0.01:
                repeat = True
            max_diff = max(max_diff, abs(output - firm['actual_output'][0]), abs(emission - firm['emission'][0]))
            firm['actual_output'] = output
            firm['emission'] = emission
            firm['profit'] = profit
            if(print_output):
                print("Firm {} has output: {:5f} and emission: {:5f} and profit: {:2f}".format(firm['name'][0], firm['actual_output'][0], firm['emission'][0], profit))
        if(print_diff): 
            sys.stdout.write("\rMax diff: {:5f}".format(max_diff))
            sys.stdout.flush()
optimize_them_all(firms, dummy_market, dummy_sector)


The profit expr: -2*em + out*(10 - 0.1*out) + 0.2*out - 5*(-em + out)**2
Firm dummy_firm1 has output: 41.000000 and emission: 40.800000 and profit: 168.300000000000
The profit expr: -2*em + out*(5.89999999999065 - 1.1*out) + 0.2*out - 0.01*(-em + out)**2
Firm dummy_firm2 has output: 2.747748 and emission: 0.000000 and profit: 8.38063063043580
The profit expr: -2*em + out*(9.72522522520855 - 0.1*out) + 0.2*out - 5*(-em + out)**2
Firm dummy_firm1 has output: 39.626126 and emission: 39.426126 and profit: 157.222987175705
The profit expr: -2*em + out*(6.03738738739054 - 1.1*out) + 0.2*out - 0.01*(-em + out)**2
Firm dummy_firm2 has output: 2.809634 and emission: 0.000000 and profit: 8.76238770712769
The profit expr: -2*em + out*(9.71903660415794 - 0.1*out) + 0.2*out - 5*(-em + out)**2
Firm dummy_firm1 has output: 39.595183 and emission: 39.395183 and profit: 156.977851844983
The profit expr: -2*em + out*(6.04048169791447 - 1.1*out) + 0.2*out - 0.01*(-em + out)**2
Firm dummy_firm2 has output

In [10]:
# Select random seed
import numpy as np
np.random.seed(0)

#Let's create a function that will generate the dataframes for the market, sector and 10 firms
x = sp.symbols('x')
def generate_dataframes(number_of_firms = 10):
    #Generate the market
    market = pd.DataFrame({
        'id': 1,
        'name': 'dummy_market',
        'permit_price': [5] # the price is fixed and is the market-clrearing price 
        # If permit price < 2.3 then it crushes
        })
    #Generate the sector
    sector = pd.DataFrame({
    'id': 1,
        'name': 'dummy_sector',
        'price_demand_function': [100 - 0.1*x]
        })
    #Generate the firms
    firms = []
    for i in range(number_of_firms):
        firm = pd.DataFrame({
            'id': i,
            'name': f'dummy_firm{i}',
            'sector_id': 1,
            'country': 'US',
            'production_cost_function': [x*0],
            'abatement_cost_function': [np.random.rand()*0.5 *x**2],
            'emission_to_output_ratio': 1,
            'actual_output': np.random.randint(1, 2),
            'emission': 0,
            'free_emission_multiplier': [np.random.rand()],
            'profit': 0
            })
        firms.append(firm)
    return market, sector, firms
market, sector, firms = generate_dataframes()

for firm in firms:
    
    firm["actual_output"] = np.random.randint(100, 20000)
optimize_them_all(firms, market, sector)

#Save the firms dataframe to a csv file
firms_df = pd.concat(firms)
firms_df.to_csv('firms2.csv', index=False)







The profit expr: -5*em + out*(-0.1*out - 10233.1) + 3.5759468318621*out - 0.274406751963662*(-em + out)**2
Firm dummy_firm0 has output: 0.000000 and emission: 0.000000 and profit: -0.000000690479465362813
The profit expr: -5*em + out*(-0.1*out - 9203.10000000001) + 2.72441591498448*out - 0.301381688035822*(-em + out)**2
Firm dummy_firm1 has output: 0.000000 and emission: 0.000000 and profit: -0.00000118518189861212
The profit expr: -5*em + out*(-0.1*out - 9117.60000000002) + 3.22947056533328*out - 0.211827399669452*(-em + out)**2
Firm dummy_firm2 has output: 0.000000 and emission: 0.000000 and profit: -0.00000114318452001500
The profit expr: -5*em + out*(-0.1*out - 9027.90000000003) + 4.4588650039104*out - 0.218793605631346*(-em + out)**2
Firm dummy_firm3 has output: 0.000000 and emission: 0.000000 and profit: -0.00000118811310675439
The profit expr: -5*em + out*(-0.1*out - 7313.60000000005) + 1.91720759412889*out - 0.481831380250515*(-em + out)**2
Firm dummy_firm4 has output: 0.000000

In [220]:
# Robustsness check for the initial value of the output
def robustness_check():
    # Select random seed
    np.random.seed(0)
    market, sector, firms = generate_dataframes(100)
    optimize_them_all(firms, market, sector, print_output=False, print_diff=True)
    firms_df = pd.concat(firms)
    firms_df.to_csv('firms100.csv', index=False)

    # Select random seed
    for i in tqdm.tqdm(range(10)):
        np.random.seed(i)
        for firm in firms:
            firm["actual_output"] = np.random.randint(100, 20000)
        optimize_them_all(firms, market, sector, print_output=False, print_diff=True)
        # Compare to firms100.csv and print the diviation on emission, output and profit
        firms_df = pd.concat(firms)
        firms100 = pd.read_csv('firms100.csv')
        print("Seed: {}".format(i))
        # Calculate the max and mean diviation of profit, emissions and output, between the two runs
        max_output_diff = 0
        mean_output_diff = 0
        max_emission_diff = 0
        mean_emission_diff = 0
        max_profit_diff = 0
        mean_profit_diff = 0
        for i in range(len(firms)):
            max_output_diff = max(max_output_diff, abs(firms[i]['actual_output'][0] - firms100['actual_output'][i]))
            mean_output_diff += abs(firms[i]['actual_output'][0] - firms100['actual_output'][i])
            max_emission_diff = max(max_emission_diff, abs(firms[i]['emission'][0] - firms100['emission'][i]))
            mean_emission_diff += abs(firms[i]['emission'][0] - firms100['emission'][i])
            max_profit_diff = max(max_profit_diff, abs(firms[i]['profit'][0] - firms100['profit'][i]))
            mean_profit_diff += abs(firms[i]['profit'][0] - firms100['profit'][i])
        mean_output_diff /= len(firms)
        mean_emission_diff /= len(firms)
        mean_profit_diff /= len(firms)
        print("Max output diff: {:5f}".format(max_output_diff), end = " ")
        print("Mean output diff: {:5f}".format(mean_output_diff), end = " ")
        print("Max emission diff: {:5f}".format(max_emission_diff), end = " ")
        print("Mean emission diff: {:5f}".format(mean_emission_diff), end = " ")
        print("Max profit diff: {:5f}".format(max_profit_diff), end = " ")
        print("Mean profit diff: {:5f}".format(mean_profit_diff), end = " ")
        print("")
#robustness_check()


